Translate to the following languages:
- en
- tl
- vi
- th
- zh

for zero-shot languages:
- bn
- fa
- gu
- mr
- pa
- no
- si
- sq
- ru
- az
- pt
- nl
- fr

In [1]:
target_langs = ["en", "tl", "vi", "th", "zh"]

In [2]:
from google.cloud import translate

# Initialize Translation client
def translate_text(
    text: list[str] = ["YOUR_TEXT_TO_TRANSLATE"], project_id: str = "YOUR_PROJECT_ID", target_language_code: str = "tl"
) -> translate.TranslationServiceClient:
    """Translating Text."""

    client = translate.TranslationServiceClient()

    location = "global"

    parent = f"projects/{project_id}/locations/{location}"

    # Translate text from English to French
    # Detail on supported types can be found here:
    # https://cloud.google.com/translate/docs/supported-formats
    response = client.translate_text(
        request={
            "parent": parent,
            "contents": text,
            "mime_type": "text/plain",  # mime types: text/plain, text/html
            "target_language_code": target_language_code,
        }
    )

    return response

In [6]:
import pandas as pd

train_df = pd.read_csv("xlm_fakenews/train_augmented.tsv", sep="\t")


In [9]:
train_df.drop(columns=["augmented_en"], inplace=True)

In [5]:
train_df = train_df[[1, 2]]

In [7]:
train_df.columns = ["label", "text"]

In [9]:
train_list = train_df["text"].to_list()

In [10]:
from torch.utils.data import DataLoader

In [11]:
batcher = DataLoader(train_list, batch_size=32, shuffle=False)

In [12]:
from tqdm import tqdm

for lang in target_langs:
    augmented_texts = []
    for batch in tqdm(batcher):
        translated_text = translate_text(batch, "trusty-moment-407614", lang)
        texts = [translation.translated_text for translation in translated_text.translations]
        augmented_texts.extend(texts)

    train_df['augmented_'+lang] = augmented_texts

100%|██████████| 320/320 [04:19<00:00,  1.24it/s]
/var/folders/qp/m74hqv_10t3ckj5rm579lm3m0000gp/T/ipykernel_8172/2412749338.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['augmented_'+lang] = augmented_texts
100%|██████████| 320/320 [04:12<00:00,  1.27it/s]
/var/folders/qp/m74hqv_10t3ckj5rm579lm3m0000gp/T/ipykernel_8172/2412749338.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['augmented_'+lang] = augmented_texts
100%|██████████| 320/320 [04:13<00:00,  1.26it/s]
/var/folders/q

In [10]:
train_df.to_csv("xlm_fakenews/train_augmented.tsv", sep="\t", index=False)

In [11]:
train_df.head()

,label,text,augmented_tl,augmented_vi,augmented_th,augmented_zh
0,false,Says the Annies List political group supports ...,Sabi ng grupong pampulitika ng Annies List ay ...,Nói rằng nhóm chính trị Annies List ủng hộ việ...,กลุ่มการเมือง Annies List สนับสนุนการทำแท้งในไ...,安妮名单政治团体表示支持妊娠晚期按需堕胎。
1,half-true,When did the decline of coal start? It started...,Kailan nagsimula ang pagbaba ng karbon? Nagsim...,Sự suy giảm của than bắt đầu từ khi nào? Nó bắ...,การลดลงของถ่านหินเริ่มต้นเมื่อใด? มันเริ่มต้นข...,煤炭的衰落是从什么时候开始的？它始于天然气腾飞，始于（总统乔治·W·布什）政府。
2,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",Sumasang-ayon si Hillary Clinton kay John McCa...,"Hillary Clinton đồng ý với John McCain ""bằng c...","ฮิลลารี คลินตันเห็นด้วยกับจอห์น แมคเคน ""ด้วยกา...",希拉里·克林顿同意约翰·麦凯恩的观点，“投票支持乔治·布什在伊朗问题上无罪推定”。
3,false,Health care reform legislation is likely to ma...,Ang batas sa reporma sa pangangalagang pangkal...,Luật cải cách chăm sóc sức khỏe có thể sẽ yêu ...,กฎหมายปฏิรูปการดูแลสุขภาพมีแนวโน้มที่จะกำหนดให...,医疗保健改革立法可能会强制要求免费进行变性手术。
4,half-true,The economic turnaround started at the end of ...,Nagsimula ang economic turnaround sa pagtatapo...,Sự thay đổi kinh tế bắt đầu vào cuối nhiệm kỳ ...,การพลิกฟื้นทางเศรษฐกิจเริ่มต้นเมื่อสิ้นสุดภาคเ...,经济好转是在我任期结束时开始的。
